# Projeto de Análise de Dados - IMDB

## Objetivo do Notebook 2

Neste notebook, realizei a etapa da extração e upload dos dados brutos do projeto. Os arquivos originais, no formato TSV, foram carregados para o ambiente do Databricks e estruturados como DataFrames. Também foram criadas as primeiras tabelas no Metastore para facilitar consultas posteriores via SQL.


## 1 - Importação das bibliotecas PySpark:

In [0]:

from pyspark.sql import SparkSession

## 2 - Limpeza do ambiente antes da carga dos dados para garantir um ambiente limpo e sem resíduos de execuções anteriores:

In [0]:
tables = [
    "name_basics", 
    "title_akas", 
    "title_basics", 
    "title_crew", 
    "title_episode", 
    "title_ratings",
    "title_principals"
]

# Remover diretórios existentes no warehouse
for table in tables:
    path = f"dbfs:/user/hive/warehouse/{table}"
    dbutils.fs.rm(path, True)

print("Limpeza inicial concluída ✅")


Limpeza inicial concluída ✅


## 3 - Upload e Leitura dos Dados

In [0]:
def ler_tsv(caminho):
    return spark.read.option("header", "true").option("sep", "\t").option("inferSchema", "true").csv(caminho)

name_basics = ler_tsv("/FileStore/tables/name_basics_tsv.gz")
title_akas = ler_tsv("/FileStore/tables/title_akas_tsv.gz")
title_basics = ler_tsv("/FileStore/tables/title_basics_tsv.gz")
title_crew = ler_tsv("/FileStore/tables/title_crew_tsv.gz")
title_episode = ler_tsv("/FileStore/tables/title_episode_tsv.gz")
title_ratings = ler_tsv("/FileStore/tables/title_ratings_tsv.gz")
title_principals = ler_tsv("/FileStore/tables/title_principals_tsv.gz")

print("Leitura dos dados concluída ✅")


Leitura dos dados concluída ✅


## 4 - Verificações iniciais: contagem das linhas

In [0]:

print("Quantidade de registros por tabela:")
print(f"name_basics: {name_basics.count()}")
print(f"title_akas: {title_akas.count()}")
print(f"title_basics: {title_basics.count()}")
print(f"title_crew: {title_crew.count()}")
print(f"title_episode: {title_episode.count()}")
print(f"title_ratings: {title_ratings.count()}")
print(f"title_principals: {title_principals.count()}")

Quantidade de registros por tabela:
name_basics: 14322508
title_akas: 51851599
title_basics: 11576566
title_crew: 11576566
title_episode: 8906003
title_ratings: 1555765
title_principals: 91920801


## 5 - Criação das Temp Views

Após a leitura dos arquivos e verificações iniciais, criei visualizações temporárias (Temp Views), para consultar os dados via SQL nos próximos notebooks.


In [0]:
name_basics.createOrReplaceTempView("name_basics")
title_akas.createOrReplaceTempView("title_akas")
title_basics.createOrReplaceTempView("title_basics")
title_crew.createOrReplaceTempView("title_crew")
title_episode.createOrReplaceTempView("title_episode")
title_ratings.createOrReplaceTempView("title_ratings")
title_principals.createOrReplaceTempView("title_principals")


##6 - Remoção de tabelas e diretórios antigos e criação das tabelas



In [0]:
# DROP de tabelas existentes no metastore
spark.sql("DROP TABLE IF EXISTS name_basics")
spark.sql("DROP TABLE IF EXISTS title_akas")
spark.sql("DROP TABLE IF EXISTS title_basics")
spark.sql("DROP TABLE IF EXISTS title_crew")
spark.sql("DROP TABLE IF EXISTS title_episode")
spark.sql("DROP TABLE IF EXISTS title_ratings")
spark.sql("DROP TABLE IF EXISTS title_principals")

# Limpeza dos diretórios
dbutils.fs.rm("dbfs:/user/hive/warehouse/name_basics", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/title_akas", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/title_basics", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/title_crew", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/title_episode", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/title_ratings", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/title_principals", True)

# Criação das tabelas
name_basics.write.mode("overwrite").parquet("/FileStore/parquet/name_basics")
title_akas.write.mode("overwrite").parquet("/FileStore/parquet/title_akas")
title_basics.write.mode("overwrite").parquet("/FileStore/parquet/title_basics")
title_crew.write.mode("overwrite").parquet("/FileStore/parquet/title_crew")
title_episode.write.mode("overwrite").parquet("/FileStore/parquet/title_episode")
title_ratings.write.mode("overwrite").parquet("/FileStore/parquet/title_ratings")
title_principals.write.mode("overwrite").parquet("/FileStore/parquet/title_principals")

print("Salvamento em Parquet concluído ✅")

# Criação das tabelas no metastore a partir dos Parquet salvos
for table in tables:
    spark.sql(f"CREATE TABLE {table} USING parquet LOCATION '/FileStore/parquet/{table}'")

print("Tabelas criadas no Metastore ✅")


Salvamento em Parquet concluído ✅
Tabelas criadas no Metastore ✅


## 7 - Verificação das tabelas no metastore e seus schemas



In [0]:
# Verificação de todas as tabelas existentes no metastore
spark.sql("SHOW TABLES").show()

# Verificação dos schemas
tables = [
    "name_basics", 
    "title_akas", 
    "title_basics", 
    "title_crew", 
    "title_episode", 
    "title_ratings",
    "title_principals"
]

for table in tables:
    print("=========================================")
    print(f"📋 Schema simplificado da tabela: {table}")
    print("=========================================")
    spark.table(table).printSchema()
    print("\n\n")


 


+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
| default|     name_basics|      false|
| default|      title_akas|      false|
| default|    title_basics|      false|
| default|      title_crew|      false|
| default|   title_episode|      false|
| default|title_principals|      false|
| default|   title_ratings|      false|
+--------+----------------+-----------+

📋 Schema simplificado da tabela: name_basics
root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)




📋 Schema simplificado da tabela: title_akas
root
 |-- titleId: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-

## Conclusão

Finalizada a etapa de extração e upload dos dados.

Nos próximos notebooks, vamos realizar a preparação dos dados na camada silver (notebook3) e realizar consultas analíticas com os dados já trabalhados, na camada gold (notebook4).
